In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json
import re

consumer_key = '7iBtytyrtytrK5c9t3dDfzgF'
consumer_secret = 'lnC2IIvXrs8qiPshSlo73OMe7v7NKTIXOu3402ODtL0Lp0uFzh'
access_token = '912715498810257408-aoO3owMw5lTQoqxNrN8pVGWgzrvyZUW'
access_secret = 'gTFRWYgYCfU3X2JGOQ4hW79W779FqWhCAHwsGC3dsWYmf'

def clean_tweet(tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

class TweetsListener(StreamListener):

  def __init__(self, csocket):
      self.client_socket = csocket

  def on_data(self, data):
      try:
          msg = json.loads( data )
          #print( msg['text'].encode('utf-8') )
          tweet_text = clean_tweet(msg['text'])
          #print(tweet_text)
          self.client_socket.send( tweet_text + '\n' )
          return True
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      return True

  def on_error(self, status):
      print(status)
      return True

def sendData(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)

  twitter_stream = Stream(auth, TweetsListener(c_socket))
  #Use the below line to get whole world's trending topics
  #twitter_stream.filter(locations=[-180,-90,180,90])
  #Due to too much of tweets volume and execution time, I am using the below line to get output
  #twitter_stream.filter(track=['iphone'])
  twitter_stream.sample()


if __name__ == "__main__":
  s = socket.socket()         # Create a socket object
  host = "192.0.0.1"      # Get local machine name
  port = 7799                 # Reserve a port for your service.
  s.bind((host, port))        # Bind to the port

  print("Listening on port: %s" % str(port))

  s.listen(5)                 # Now wait for client connection.
  c, addr = s.accept()        # Establish connection with client.

  print( "Received request from: " + str( addr ) )

  sendData( c )